In [2]:
import openai
from pygame import mixer
from io import BytesIO
from gtts import gTTS
import speech_recognition as sr
import pyaudio
import pyttsx3
import os


while True:
    Lang = input("Please choose the language:zh for Chinese,en for English ")
    if Lang == 'zh':
        language = 'cmn-Hans-CN'
        break
    elif Lang == 'en':
        language = 'en'
        break
    else:
        print('Sorry,I have not learnt that language!')
        
def get_speak(response,Lang):
    mp3_fp = BytesIO()
    tts = gTTS(text=response.choices[0].message.content, lang=Lang)  
    tts.write_to_fp(mp3_fp)
    return mp3_fp


while True:
    print('Get ready!Recording is starting...') 
    r = sr.Recognizer()
    with sr.Microphone() as source:                   
        r.adjust_for_ambient_noise(source,duration = 1.1) 
        audio = r.listen(source)
    try:
        text = r.recognize_google(audio, language=language)
        print('You said:',text)                     
        judge = input('Is it correct? if Yes, press Y or y, otherwise press N or n: ')
        if judge in ['Y','y','Yes','yes']:
            right_question = text
        if judge in ['N','n','No','no']:
            right_question = input('Please input the right question: ')
            if right_question in ['N','n','No','no']:
                break
            
    except LookupError:                            # speech is unintelligible
        print("Could not understand audio")
    
    with open('Key.txt','r') as f:
        openai.api_key = f.read()
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=[{"role":"user", 
                                                                    "content": right_question}])
    # Play automatically
    mixer.init()
    sound = get_speak(response,Lang)
    sound.seek(0)
    mixer.music.load(sound, "mp3")
    mixer.music.play()
      
    while mixer.music.get_busy():  
        pause = input('If you want to pause, please enter "p" or "P" ')
        if pause in ['p','P']:
            mixer.music.pause()
            conti = input('Do you want to continue playing? if yes, enter y, else enter n: ')
            if conti in ['Y','y','Yes','yes']:
                mixer.music.unpause()
            if conti in ['N','n','No','no']:
                mixer.music.stop()
        
    go_on = input('Audio is over.Do you want to go on?')
    if go_on in ['Y','y','Yes','yes']:
        continue
    elif go_on in ['N','n','No','no']:
        mixer.music.stop()
        break

engine = pyttsx3.init() 
voices = engine.getProperty('voices')
engine.setProperty("voice", voices[2].id)  
engine.setProperty("rate", 130)            
engine.say('Goodbye')
engine.runAndWait()


RuntimeError: Dynamic linking causes SDL downgrade! (compiled with version 2.26.1, linked to 2.24.2)